# 框架的使用和示例：线性回归数据

此笔记本是一个线性回归问题的例子，用来帮助用户理解和掌握如何使用训练器框架。

In [1]:
import sys 
sys.path.append("../..")

from core.trainer import Trainer


## 0. 训练器实例

训练器 `core.trainer.Trainer` 实例化时，需指定以下参数：
- exp_name: 实验名称
- epoch: 本次训练轮数
- batch_size: 批数据数量
- gradient_accumulation_step(=1): 梯度累积步数
- init_random_seed(=None): 初始随机种子
- device(="cpu"): 训练设备
- enable_auto_mixed_precision(=True): 自动混合精度
- log_tool(="tensorboard"): 日志工具
- log_dir(=None): 日志目录

In [2]:
trainer = Trainer(
    exp_name="lin-reg",
    epoch=20,
    batch_size=40,
    # gradient_accumulation_step=1,
    
    init_random_seed=0,
    device="cuda:2",
    # enable_auto_mixed_precision=True,
    
    # log_tool="tensorboard",
    # log_dir="../tb_log",
)

## 1. 数据集

数据集为通用的 torch.utils.data.Dataset 类，但在使用本训练框架时，必须为数据集类实现 `__len__` 方法，否则模型将无法定义可复现的数据加载器。

在本例子中，我们定义如下的线性数据集 `LinearData`，其数据保存在 `data.pt` 文件中，为随机生成的正太随机数所给出的线性模型（截距为 0）:

In [3]:
import torch 
from torch.utils.data import Dataset 

class LinearData(Dataset):
    def __init__(self, split="train") -> None:
        super().__init__()
        data = torch.load("data.pt")[split]
        self.x = data["x"]
        self.y = data["y"]
    
    def __getitem__(self, index):
        return self.x[index], self.y[index]
    
    def __len__(self):
        return len(self.x)

dataset = LinearData()

## 2. 神经网络模型（torch.nn.Module）

神经网络模型为通用的 `torch.nn.Module` 类。

在本例子中，我们使用一个简单的线性层:

In [4]:
import torch.nn as nn 

network = nn.Linear(10, 1, bias=True)

## 3. 损失函数计算

损失函数通过元组的形式传入训练器的`train`函数，其打包格式为：

- ([name, ...], [loss_fn, ...])

或

- ([name, ...], [loss_fn, ...], [loss_weight, ...])

在使用第一种打包格式时，各 loss 的计算权重默认为 1.0，损失函数的名称、计算函数、计算权重为一一对应关系。其中，`loss_fn` 接收的输入同一为两个位置参数：network 和 batch，分别对应神经网络和批数据。

在本例子中，我们使用 MSE 作为损失函数:

In [5]:
mse = nn.MSELoss()

def mse_loss_fn(network, batch):
    x, y = batch 
    y_hat = network(x).squeeze()
    return mse(y, y_hat)

losses = (
    ["mse"],
    [mse_loss_fn]
)

## 4. 优化器部署函数（Optional）

在每次训练中，训练器将部署一个新的优化器实例，实例化优化器由 `optim_fn` 这一参数实现。`optim_fn` 参数为一个函数，它接收一个神经网络 `network` 作为唯一输入，并返回一个优化器实例。在框架中，默认的 `optim_fn` 会返回一个默认的 `AdamW` 优化器。

在本例子中，我们新定义一个优化器部署函数，它将在训练中使用 SGD 优化器，并使用 0.2 的学习率：

In [6]:
from torch.optim import SGD 

def sgd_optim_fn(network):
    return SGD(network.parameters(), lr=0.2)

## 6. 通过插件配置训练器功能

使用 `core.plugin` 中的插件，为训练器添加功能

In [7]:
from core.plugin import EpochSavePlugin, LossLoggerPlugin

trainer.extend_plugins([
    EpochSavePlugin("../checkpoints", 5),
    LossLoggerPlugin(1)
])

trainer.train(
    dataset=dataset,
    network=network,
    losses=losses,
    optim_fn=sgd_optim_fn
)

[PLUGIN](ReinitNetworkWeights) Re-initialize network weights randomly.
[PLUGIN](EpochSave) Saving trainer to '../checkpoints/epoch-5/trainer_state_dict.pth'
[PLUGIN](EpochSave) Saving network to '../checkpoints/epoch-5/network_state_dict.pth'
[PLUGIN](EpochSave) Saving optimizer to '../checkpoints/epoch-5/optimizer_state_dict.pth'
[PLUGIN](EpochSave) Saving scaler to '../checkpoints/epoch-5/scaler_state_dict.pth'
[PLUGIN](EpochSave) Saving trainer to '../checkpoints/epoch-10/trainer_state_dict.pth'
[PLUGIN](EpochSave) Saving network to '../checkpoints/epoch-10/network_state_dict.pth'
[PLUGIN](EpochSave) Saving optimizer to '../checkpoints/epoch-10/optimizer_state_dict.pth'
[PLUGIN](EpochSave) Saving scaler to '../checkpoints/epoch-10/scaler_state_dict.pth'
[PLUGIN](EpochSave) Saving trainer to '../checkpoints/epoch-15/trainer_state_dict.pth'
[PLUGIN](EpochSave) Saving network to '../checkpoints/epoch-15/network_state_dict.pth'
[PLUGIN](EpochSave) Saving optimizer to '../checkpoints/epo